In [ ]:
%matplotlib ipympl
import matplotlib as mpl
import matplotlib.pyplot as plt
import embag_python
import common.liegroups.se2_python as se2
from scipy.spatial.transform import Rotation as R
import experimental.beacon_sim.ekf_slam_python as esp
import experimental.beacon_sim.beacon_potential_python as bpp
from experimental.beacon_sim import beacon_potential_pb2
from experimental.beacon_sim import world_map_config_pb2
import common.time.robot_time_python as rtp
import planning.probabilistic_road_map_python as prmp
import experimental.beacon_sim.correlated_beacons_python as cbp
import numpy as np
import functools

In [ ]:
bag_file = '/home/erick/code/robot-spot/6th_floor_mapping_tethered_estimator.bag'


In [ ]:
def frame_from_child_at_time_from_ros_msg(ros_msg):
    # Convert the time to a float
    timestamp = ros_msg['header']['stamp'].to_sec()

    rot_vec = ros_msg["transform"]["rotation"]
    trans_vec= ros_msg["transform"]["translation"]
    parent_from_child_rot = R.from_quat([rot_vec['x'], rot_vec['y'], rot_vec['z'], rot_vec['w']])
    parent_from_child_trans = np.array([trans_vec['x'], trans_vec['y'], trans_vec['z']])

    x_axis_in_parent = parent_from_child_rot.apply(np.array([1.0, 0.0, 0.0]))
    theta = np.arctan2(x_axis_in_parent[1], x_axis_in_parent[0])

    parent_from_child = se2.SE2(theta, parent_from_child_trans[:2])
    return timestamp, parent_from_child 


In [ ]:
def estimate_from_msg(ros_msg):
    data = bytes(ros_msg.data()["estimate_proto"])
    return esp.EkfSlamEstimate.from_proto_string(data)

In [ ]:
bag = embag_python.View(bag_file)
print(bag)

maps = []
map_from_robot = []
detections = []

for msg in bag.getMessages():
    if msg.topic == '/map':
        maps.append((msg.data()["header"]["stamp"].to_sec(), msg))

    if msg.topic == "/tf":
        for transform in msg.data()["transforms"]:
            if transform["header"]["frame_id"] not in ['map', 'flat_body']:
                continue
            if transform["child_frame_id"] not in ['map', 'flat_body']:
                continue
            time, parent_from_child = frame_from_child_at_time_from_ros_msg(transform)
            if transform["header"]["frame_id"] == "flat_body":
                map_from_robot_at_time = (time, parent_from_child.inverse())
            else:
                map_from_robot_at_time = (time, parent_from_child)

            map_from_robot.append(map_from_robot_at_time)
    if 'detections' in msg.topic:
        detections.append((msg.data()["header"]["stamp"].to_sec(), msg))

last_map_msg = maps[-1][1]


In [ ]:
estimate = estimate_from_msg(last_map_msg)

In [ ]:
def plot_map(estimate):
    ids = estimate.beacon_ids
    xs = []
    ys = []
    for i in ids:
        beacon_in_map = estimate.beacon_in_local(i)
        xs.append(beacon_in_map[0])
        ys.append(beacon_in_map[1])
        plt.text(beacon_in_map[0], beacon_in_map[1], f'tag_{i}')
    plt.plot(xs, ys, 'o')

def plot_trajectory(map_from_robot_at_time):
    for _, map_from_robot in map_from_robot_at_time[::5]:
        arrow_start = (map_from_robot @ np.array([0, 0])).flatten()
        x_axis = (map_from_robot @ np.array([1, 0])).flatten()
        delta = x_axis - arrow_start
        plt.arrow(arrow_start[0], arrow_start[1], delta[0], delta[1], head_width=0.25)

def plot_road_map(road_map):
    pts = np.stack(road_map.points())
    plt.plot(pts[:, 0], pts[:, 1], '*')

    for i, pt in enumerate(pts):
        plt.text(*pt, str(i))

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])
    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)
    ...

In [ ]:
road_map_pts = [
    np.array([[0.0], [-0.6]]),
    np.array([[2.8], [0.0]]),
    np.array([[18.3], [1.65]]),
    np.array([[38.2], [4.5]]),
    np.array([[53.6], [6.65]]),
    np.array([[56.5], [7.2]]),
    np.array([[0.8], [16.0]]),
    np.array([[-2.5], [15.9]]),
    np.array([[16.16], [17.8]]),
    np.array([[36.0], [20.2]]),
    np.array([[51.2], [22.9]]),

]

adj = [
    (0, 1),
    (1, 2), 
    (2, 3), 
    (3, 4), 
    (4, 5),
    (4, 10),
    (10, 9),
    (9, 8),
    (8, 6),
    (6, 7),
    (9, 3),
    (2, 8),
    (1, 6),
]

adj_mat = np.zeros((12, 12))

for pair in adj:
    adj_mat[pair[0], pair[1]] = 1.0
    adj_mat[pair[1], pair[0]] = 1.0
    
road_map = prmp.RoadMap(road_map_pts, adj_mat, None)

In [ ]:
all_landmarks = set(range(50, 81))
exclusive_landmarks = set([58, 64])
starting_landmarks = set([50, 51, 52, 77])
absent_landmarks = all_landmarks - exclusive_landmarks - starting_landmarks

exclusive_landmarks = cbp.BeaconClique(0.5, 1e-9, members = list(exclusive_landmarks))
exclusive_potential = cbp.create_correlated_beacons(exclusive_landmarks)
starting_potential = bpp.BeaconPotential.correlated_beacon_potential(p_present=1.0 - 1e-9, p_beacon_given_present=1.0 - 1e-9, members=list(starting_landmarks))

absent_potential = bpp.BeaconPotential.correlated_beacon_potential(p_present=1e-9, p_beacon_given_present=1e-9, members=list(absent_landmarks))

beacon_potential = exclusive_potential * absent_potential * starting_potential
beacon_potential = beacon_potential.conditioned_on({x: False for x in absent_landmarks})

In [ ]:
beacon_potential.log_prob({58: False, 64: True}, allow_partial_assignment=True)


In [ ]:
beacon_potential.log_marginals([58, 64])

In [ ]:
plt.figure()
plot_map(estimate)
# plot_trajectory(map_from_robot)
plot_road_map(road_map)

plt.axis('equal')

In [ ]:
deltas = []
for i in range(len(map_from_robot) - 1):
    _, map_from_start = map_from_robot[i]
    _, map_from_end = map_from_robot[i+1]
    start_from_end = map_from_start.inverse() * map_from_end
    deltas.append(start_from_end.translation())

deltas = np.vstack(deltas)

plt.figure()
times = [x[0] for x in map_from_robot[:-1]]
diffs = np.linalg.norm(deltas, axis=1)
plt.plot(times, diffs)
start = 1710966142.5
length = 1.2

In [ ]:
with open('/tmp/6th_floor_road_map.pb', 'wb') as file_out:
    file_out.write(road_map.to_proto_string())

with open('/tmp/6th_floor_beacon_potential.pb', 'wb') as file_out:
    file_out.write(beacon_potential.to_proto_string())

config = esp.EkfSlamConfig(
    max_num_beacons=40,
    initial_beacon_uncertainty_m=100.0,
    along_track_process_noise_m_per_rt_meter=0.1,
    cross_track_process_noise_m_per_rt_meter=0.01,
    pos_process_noise_m_per_rt_s=0.01,
    heading_process_noise_rad_per_rt_meter=0.001,
    heading_process_noise_rad_per_rt_s=0.00001,
    beacon_pos_process_noise_m_per_rt_s=0.01,
    range_measurement_noise_m=0.05,
    bearing_measurement_noise_rad=0.005,
    on_map_load_position_uncertainty_m=0.1,
    on_map_load_heading_uncertainty_rad=0.01,
)
time = rtp.RobotTimestamp()
slam = esp.EkfSlam(config, time)
slam.estimate = estimate
slam.save_map('/tmp/6th_floor_map.pb')


In [ ]:
beacons = []
for beacon_id in estimate.beacon_ids:
    beacon_in_local = estimate.beacon_in_local(beacon_id)
    beacons.append(world_map_config_pb2.Beacon(
        id=beacon_id,
        pos_x_m=beacon_in_local[0],
        pos_y_m=beacon_in_local[1],
    ))

beacon_potential_proto = beacon_potential_pb2.BeaconPotential()
beacon_potential_proto.ParseFromString(beacon_potential.to_proto_string())

correlated_beacon_config = world_map_config_pb2.CorrelatedBeaconsConfig()
correlated_beacon_config.beacons.extend(beacons)
correlated_beacon_config.potential.CopyFrom(beacon_potential_proto)

world_map_config = world_map_config_pb2.WorldMapConfig()
world_map_config.correlated_beacons.CopyFrom(correlated_beacon_config)

with open('/tmp/6th_floor_world_map_config.pb', 'wb') as file_out:
    file_out.write(world_map_config.SerializeToString())




In [ ]:
beacon_potential_proto